In [1]:
import numpy as np
import pandas as pd
import redis


In [2]:
#configure the redis db & connect to redis client
#redis-17158.c326.us-east-1-3.ec2.cloud.redislabs.com:17158
#hGTkDq5a98SkO1P6ER4KpnpXI3T9kQuy
hostname = 'redis-18301.c274.us-east-1-3.ec2.cloud.redislabs.com'
portnumber = 18301
password = 'ihIROTTqvhEMCPVmqNHPImalkOIZjZ0W'

r = redis.StrictRedis(host = hostname,
                    port = portnumber,
                    password = password)

In [3]:
r.ping()

True

In [4]:
#r.keys()

#### load the data np.zip

In [5]:
np_file = np.load('dataframe_students_teacher.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


In [6]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson@Student


In [7]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

## save data in redis - hashes
- keyname = 'academy:register'

In [8]:
#convert np array to bytes & then store bytes into hashes
for record in records:
    name_role = record['name_role'] #key
    vector = record['Facial_Features'] #value
    #print(name_role)

    # convert np arr into bytes
    vector_bytes = vector.tobytes()

    #save data into redis cloud
    r.hset(name='academy:register', key=name_role, value=vector_bytes)

In [9]:
vector.dtype

dtype('float32')

## retrieve records from redis

In [10]:
retrieve_dict = r.hgetall(name='academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series

b'Chris Evans@Student'           b')]8?\x05\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf\...
b'Angelina Jolie@Student'        b'3f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
b'Morgan Freeman@Teacher'        b'\xee\x7f\xf8>\x87\x87\x1e\xbfja\xd0\xbe\xbe\...
b'Barack Obama@Teacher'          b'5\xd6r?\xa5\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
b'Scarlett Johansson@Student'    b'K\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
dtype: object

In [11]:
#np.frombuffer(retrieve_series[b'Morgan Freeman@Teacher'], dtype=np.float32).shape
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode(), index))

In [12]:
retrieve_series.index = index

In [13]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role', 'facial_features']
retrieve_df

,name_role,facial_features
0,Chris Evans@Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
1,Angelina Jolie@Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
2,Morgan Freeman@Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
3,Barack Obama@Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
4,Scarlett Johansson@Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


#### REGISTRATION FORM

In [14]:
import cv2
from insightface.app import FaceAnalysis

In [15]:
# configure the face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


## step-1: collect person name & role

In [16]:
# python input()
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher
    
    Enter number either 1 or 2
    """)

    if role in ('1','2'):
        if role=='1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry. Try again!')
        if i == 3:
            print('Exceeds Maximum Trials.')
        
        
key = person_name+'@'+role
print('Your name = ', person_name)
print('Your role = ', role)
print('Key = ', key)

Enter your name:  Sanya Ahmed

    Please choose
    1. Student
    2. Teacher
    
    Enter number either 1 or 2
     1


Your name =  Sanya Ahmed
Your role =  Student
Key =  Sanya Ahmed@Student


In [17]:
person_name, role

('Sanya Ahmed', 'Student')

## step-2: collect samples(facial embeddings)

In [18]:
# opencv videocapture model
cap = cv2.VideoCapture(0) #0 default camera , 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
        
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2  = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 1)

        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample >= 200:
        break
        
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'): # this will trigger only when letter q(lowercase q) is pressed in keboard
        break
cap.release()
cv2.destroyAllWindows()

In [19]:
len(face_embeddings)

200

## step-3: optimize data & save in redis db

In [20]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [21]:
# convert it into bytes
x_mean_bytes = x_mean.tobytes()

In [22]:
# save key & facial bytes into redis db
r.hset(name='academy:register', key=key, value=x_mean_bytes)

1